Привет!

Это ноутбук-бейзлайн к конкурсу на каггл: https://www.kaggle.com/t/de1dfe58de894e5f915bbf6749ae2f84

Вам нужно постараться улучшить его, чтобы скор на каггле стал побольше =)
Тренировочные и тестовые данные лежат рядом с ноутбуком или их можно скачать с каггла.

In [157]:
import numpy as np
import pandas as pd

## 1. Get data

Загрузим данные и посмотрим на них:

In [158]:
train_data = pd.read_csv("train.csv")
train_data.head()

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,25,UNM,SPC,26389.0,10,1.32,0.584105,SCH,ОБЛ КУРСКАЯ,35000.0,2.0,0.0,1
1,2,F,37,MAR,SPC,19588.0,12,1.43,0.718935,SCH,РЕСПУБЛИКА ТАТАРСТАН,15000.0,0.0,0.0,1
2,3,F,28,UNM,SPC,53669.0,18,1.10,0.586015,GRD,МОСКВА Г,70000.0,4.0,0.0,1
3,4,M,34,MAR,SPC,26349.0,10,1.43,0.655703,SCH,СВЕРДЛОВСКАЯ ОБЛАСТЬ,42500.0,4.0,0.0,0
4,5,F,43,MAR,UMN,11589.0,10,1.10,0.271893,GRD,РЯЗАНСКАЯ ОБЛАСТЬ,20000.0,3.0,0.0,0


In [159]:
X_train, y_train = train_data.drop("open_account_flg", axis=1), train_data["open_account_flg"]

## 2. Preprocessing

In [160]:
train_data.isna().any()

client_id               False
gender                  False
age                     False
marital_status          False
job_position            False
credit_sum              False
credit_month            False
tariff_id               False
score_shk               False
education               False
living_region            True
monthly_income          False
credit_count             True
overdue_credit_count     True
open_account_flg        False
dtype: bool

#### fill nans in "living region"

In [161]:
X_train["living_region"].value_counts()

ОБЛ МОСКОВСКАЯ                              8564
КРАСНОДАРСКИЙ КРАЙ                          5176
САНКТ-ПЕТЕРБУРГ                             3968
МОСКВА                                      3821
ТАТАРСТАН РЕСП                              3635
РЕСП БАШКОРТОСТАН                           3358
ОБЛ ИРКУТСКАЯ                               2863
СВЕРДЛОВСКАЯ ОБЛ                            2782
МОСКВА Г                                    2509
ОБЛ НИЖЕГОРОДСКАЯ                           2504
ОБЛ ЛЕНИНГРАДСКАЯ                           2235
РОСТОВСКАЯ ОБЛ                              2111
КРАСНОЯРСКИЙ КРАЙ                           2087
ЧЕЛЯБИНСКАЯ ОБЛ                             2047
ОБЛ САМАРСКАЯ                               2034
ОБЛ КЕМЕРОВСКАЯ                             1993
ПЕРМСКИЙ КРАЙ                               1930
САНКТ-ПЕТЕРБУРГ Г                           1866
КРАЙ СТАВРОПОЛЬСКИЙ                         1833
ОБЛ АСТРАХАНСКАЯ                            1756
ОБЛ ВОЛОГОДСКАЯ     

^ действительно, какой еще самый частый регион можно было ожидать. 

In [162]:
X_train["living_region"].fillna("ОБЛ МОСКОВСКАЯ", inplace=True)

#### fill nans in "credit_count"

In [163]:
X_train["credit_count"].value_counts()

1.0     31714
2.0     25799
0.0     18219
3.0     16707
4.0      9826
5.0      5285
6.0      2760
7.0      1345
8.0       707
9.0       326
10.0      141
11.0       90
12.0       56
13.0       25
14.0       12
15.0       10
17.0        4
19.0        3
16.0        2
18.0        1
Name: credit_count, dtype: int64

In [164]:
X_train["credit_count"].fillna(1., inplace=True)

#### fill nans in "overdue_credit_count"

In [165]:
X_train["overdue_credit_count"].value_counts()

0.0    107812
1.0      5180
2.0        36
3.0         4
Name: overdue_credit_count, dtype: int64

In [166]:
X_train["overdue_credit_count"].fillna(0., inplace=True)

#### fill nans in "monthly income"

In [167]:
X_train["monthly_income"].median()

35000.0

In [168]:
X_train["monthly_income"].fillna(X_train["monthly_income"].median(), inplace=True)

### Dealing with categorical features

#### In a baseline lets delete "living_region" feature as its categorical and has too many values)

In [169]:
X_train.drop("living_region", axis=1, inplace=True)

In [170]:
cat_features = [i[0] for i in dict(X_train.dtypes).items() if 'obj' in str(i[1])]
cat_features

['gender', 'marital_status', 'job_position', 'education']

#### Convert categirical features to numerical and binarize them

In [171]:
X_train.head()

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,monthly_income,credit_count,overdue_credit_count
0,1,M,25,UNM,SPC,26389.0,10,1.32,0.584105,SCH,35000.0,2.0,0.0
1,2,F,37,MAR,SPC,19588.0,12,1.43,0.718935,SCH,15000.0,0.0,0.0
2,3,F,28,UNM,SPC,53669.0,18,1.10,0.586015,GRD,70000.0,4.0,0.0
3,4,M,34,MAR,SPC,26349.0,10,1.43,0.655703,SCH,42500.0,4.0,0.0
4,5,F,43,MAR,UMN,11589.0,10,1.10,0.271893,GRD,20000.0,3.0,0.0


In [172]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html
X_train = pd.get_dummies(X_train, columns=cat_features)

In [173]:
X_train.head()

,client_id,age,credit_sum,credit_month,tariff_id,score_shk,monthly_income,credit_count,overdue_credit_count,gender_F,...,job_position_SPC,job_position_UMN,job_position_WOI,job_position_WRK,job_position_WRP,education_ACD,education_GRD,education_PGR,education_SCH,education_UGR
0,1,25,26389.0,10,1.32,0.584105,35000.0,2.0,0.0,0,...,1,0,0,0,0,0,0,0,1,0
1,2,37,19588.0,12,1.43,0.718935,15000.0,0.0,0.0,1,...,1,0,0,0,0,0,0,0,1,0
2,3,28,53669.0,18,1.10,0.586015,70000.0,4.0,0.0,1,...,1,0,0,0,0,0,1,0,0,0
3,4,34,26349.0,10,1.43,0.655703,42500.0,4.0,0.0,0,...,1,0,0,0,0,0,0,0,1,0
4,5,43,11589.0,10,1.10,0.271893,20000.0,3.0,0.0,1,...,0,1,0,0,0,0,1,0,0,0


## 3. Build the model

In [285]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer
from sklearn.grid_search import GridSearchCV

На каггле наши ответы будут оцениваться метрикой Mean F1-Score. Это соответствует метрике F1 из sklearn с параметром average="weighted": http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

Так как в GridSearchCV мы не можем передавать параметры метрики, то нужно сделать свою функцию метрики. Это легко делается с помощью make_scorer: в нее надо передать функцию метрики из sklearn и параметры, с которыми мы хотим ее использовать:

In [286]:
mean_f1_metric = make_scorer(f1_score, average='weighted')

Теперь мы можем использовать нашу функцию mean_f1_metric для подсчета метрики Mean F1-Score и передавать ее в GridSearchCV или cross-val-score =)

In [290]:
rf = RandomForestClassifier()

params = {
    'n_estimators' : [1, 10, 50],
    'max_depth' : [None, 7, 9]
}

gsv_rf = GridSearchCV(rf, param_grid=params, scoring=mean_f1_metric, cv=5, verbose=1)
gsv_rf.fit(X_train, y_train)
gsv_rf.best_params_, gsv_rf.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.6min finished


({'max_depth': None, 'n_estimators': 50}, 0.7633812268545622)

Натренируем на всех примерах:

In [291]:
rf = RandomForestClassifier(n_estimators=50)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## 4. Process test data

In [292]:
X_test = pd.read_csv("test.csv")
X_test.head()

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count
0,119519,F,24,MAR,SPC,28429.0,18,1.10,0.593836,GRD,ОБЛ ИРКУТСКАЯ,33000.0,3.0,0.0
1,119520,M,25,UNM,SPC,15997.0,10,1.60,0.615015,SCH,УЛЬЯНОВСКАЯ ОБЛ,35000.0,2.0,0.0
2,119521,M,25,UNM,SPC,11043.0,10,1.16,0.666758,SCH,РЕСП БАШКОРТОСТАН,25000.0,3.0,0.0
3,119522,F,34,MAR,SPC,14617.0,10,1.40,0.447745,GRD,ПЕНЗЕНСКАЯ ОБЛ,15000.0,2.0,0.0
4,119523,M,33,MAR,SPC,38147.0,12,1.60,0.706974,UGR,ОБЛ МОСКОВСКАЯ,55000.0,1.0,0.0


Нам нужно предобработать тестовые данные точно так же, как тренировочные, чтобы можно было засовывать их в модели, обученные на тренировочных данных. 

In [293]:
X_test.isna().any()

client_id               False
gender                  False
age                     False
marital_status          False
job_position            False
credit_sum              False
credit_month            False
tariff_id               False
score_shk               False
education               False
living_region            True
monthly_income           True
credit_count             True
overdue_credit_count     True
dtype: bool

In [294]:
X_test.drop("living_region", axis=1, inplace=True)

In [295]:
X_test["monthly_income"].fillna(X_train["monthly_income"].median(), inplace=True)
X_test["credit_count"].fillna(1., inplace=True)
X_test["overdue_credit_count"].fillna(0., inplace=True)

In [296]:
X_test = pd.get_dummies(X_test, columns=cat_features)

In [297]:
X_test.head()

,client_id,age,credit_sum,credit_month,tariff_id,score_shk,monthly_income,credit_count,overdue_credit_count,gender_F,...,job_position_SPC,job_position_UMN,job_position_WOI,job_position_WRK,job_position_WRP,education_ACD,education_GRD,education_PGR,education_SCH,education_UGR
0,119519,24,28429.0,18,1.10,0.593836,33000.0,3.0,0.0,1,...,1,0,0,0,0,0,1,0,0,0
1,119520,25,15997.0,10,1.60,0.615015,35000.0,2.0,0.0,0,...,1,0,0,0,0,0,0,0,1,0
2,119521,25,11043.0,10,1.16,0.666758,25000.0,3.0,0.0,0,...,1,0,0,0,0,0,0,0,1,0
3,119522,34,14617.0,10,1.40,0.447745,15000.0,2.0,0.0,1,...,1,0,0,0,0,0,1,0,0,0
4,119523,33,38147.0,12,1.60,0.706974,55000.0,1.0,0.0,0,...,1,0,0,0,0,0,0,0,0,1


## 5. Get file with answers

In [298]:
preds = rf.predict(X_test)

In [299]:
preds.mean()

0.05401975713560579

In [300]:
ans = pd.DataFrame(np.concatenate([X_test["client_id"].reshape(-1, 1), preds.reshape(-1, 1)], axis=1), columns=["client_id", "open_account_flg"])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [301]:
ans.to_csv("sampleSubmission.csv", index=None)